In [17]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

In [18]:
# Load and preprocess CelebA dataset
def load_celeba_data(img_dir, attr_file):
    if not os.path.exists(attr_file):
        raise FileNotFoundError("Attribute file not found. Ensure it is correctly placed.")
    
    attr_df = pd.read_csv(attr_file).head(350)  # Select only first 350 images
    images, labels = [], []
    
    for _, row in attr_df.iterrows():
        img_name = str(row['image_id'])  # Ensure image_id is string
        img_path = os.path.join(img_dir, img_name)
        if os.path.exists(img_path):
            img = cv2.imread(img_path)
            img = cv2.resize(img, (64, 64)) / 255.0
            gender = 1 if row['Male'] == -1 else 0  # 1 for Female, 0 for Male
            hair_length = 1 if row['Bangs'] == -1 or row['Wavy_Hair'] == 1 else 0  # 1 for long, 0 for short
            images.append(img)
            labels.append([gender, hair_length])
    return np.array(images), np.array(labels, dtype=np.int32)

In [19]:
# Paths
IMG_DIR = r"C:\Users\Shreyash D. Shinde\Downloads\archive\img_align_celeba\img_align_celeba"
ATTR_FILE = r"C:\Users\Shreyash D. Shinde\Downloads\archive\list_attr_celeba.csv"

In [20]:
# Load dataset
X, y = load_celeba_data(IMG_DIR, ATTR_FILE)

if y.shape[1] != 2:
    raise ValueError("Error: Labels are not formatted correctly.")

y_gender, y_hair = y[:, 0], y[:, 1]

In [21]:
# Build CNN model
def create_model():
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(2, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [22]:
# Train the model for gender
model_gender = create_model()
model_gender.fit(X, y_gender, epochs=10, batch_size=32)

# Train the model for hair length
model_hair = create_model()
model_hair.fit(X, y_hair, epochs=10, batch_size=32)

c:\Users\Shreyash D. Shinde\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 98ms/step - accuracy: 0.4410 - loss: 0.9341
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step - accuracy: 0.5822 - loss: 0.6783
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - accuracy: 0.6426 - loss: 0.6052
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - accuracy: 0.7478 - loss: 0.5021
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - accuracy: 0.7844 - loss: 0.4813
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - accuracy: 0.8168 - loss: 0.3589
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - accuracy: 0.8982 - loss: 0.2809
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step - accuracy: 0.9207 - loss: 0.2050
Epoch 9/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - accuracy: 0.9546 - loss: 0.1333
Epoch 10/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - accuracy: 0.9832 - loss: 0.0894
Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 102ms/step - accuracy: 0.7979 - loss: 0.5281
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step - accuracy

In [23]:
# GUI using Tkinter
def predict_image():
    file_path = filedialog.askopenfilename()
    if not file_path:
        return
    
    img = cv2.imread(file_path)
    if img is None:
        result_label.config(text="Error: Unable to load image!")
        return
    
    img = cv2.resize(img, (64, 64)) / 255.0
    img = np.expand_dims(img, axis=0)
    
    pred_gender = model_gender.predict(img)
    pred_hair = model_hair.predict(img)
    
    gender_label = "Female" if pred_gender[0][1] > pred_gender[0][0] else "Male"
    hair_label = "Long Hair" if pred_hair[0][1] > pred_hair[0][0] else "Short Hair"
    
    result_label.config(text=f"Predicted Gender: {gender_label}\nPredicted Hair Length: {hair_label}")

In [ ]:
# Tkinter Window
root = tk.Tk()
root.title("Long Hair Identification")
button = tk.Button(root, text="Upload Image", command=predict_image)
button.pack()
result_label = tk.Label(root, text="")
result_label.pack()
root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
